In [1]:
import os
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\flash\__init__.py:21: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\flash\__init__.py:21: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [3]:
#download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    input_field="review",
    target_fields="sentiment",
    train_file="../data/imdb/train.csv",
    val_file="../data/imdb/valid.csv",
    test_file="../data/imdb/test.csv",
    batch_size=3
)

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/Roma/Documents/Programming/Practical-Deep-Learning-at-Scale-with-MLFlow/chapter03/data/imdb/train.csv'

In [5]:
classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes)
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [5]:

EXPERIMENT_NAME = "dl_model_chapter03"
mlflow.set_tracking_uri('http://localhost')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [6]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test()

2021/12/04 17:11:31 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.

  | Name          | Type                          | Params
----------------------------------------------------------------
0 | train_metrics | ModuleDict                    | 0     
1 | val_metrics   | ModuleDict                    | 0     
2 | model         | BertForSequenceClassification | 4.4 M 
----------------------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Testing: 100%|█████████▉| 624/625 [00:09<00:00, 66.44it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6355999708175659, 'test_cross_entropy': 0.6506440043449402}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 625/625 [00:09<00:00, 66.64it/s]


In [7]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

run_id: e1af8b8893ca49cfbff1e1077b2b81c5; lifecycle_stage: active


In [14]:
# use the run_id to construct a logged_model URI. An example is shown here:
# logged_model = 'runs:/37a3fe9b6faf41d89001eca13ad6ca47/model'
logged_model = f'runs:/{run_id}/model'


# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)
model.predict({'What a piece of disappointing news'})


['negative']

In [12]:
# register the model
model_registry_version = mlflow.register_model(logged_model, 'nlp_dl_model')
print(f'Model Name: {model_registry_version.name}')
print(f'Model Version: {model_registry_version.version}')

Registered model 'nlp_dl_model' already exists. Creating a new version of this model...
2021/12/04 17:21:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nlp_dl_model, version 7


Model Name: nlp_dl_model
Model Version: 7


Created version '7' of model 'nlp_dl_model'.
